# BGG API Scratchpad

Help for API: https://boardgamegeek.com/wiki/page/BGG_XML_API2

Need BeautifulSoup4 and Requests libraries (through pip)

In [22]:
import requests
from bs4 import BeautifulSoup

In [2]:
def build_query(query_type: str, params: dict) -> str:
    '''Build XML query for board game

    Args:
        query_type (str): type of query: thing, user, search...
        params (dict): dict of params using str for both keys and values

    Returns:
        str: query url
    '''
    url = f'https://boardgamegeek.com/xmlapi2/{query_type}?'
    for key, value in params.items():
        url += f'&{key}={str(value)}'
    return url


def fetch_game(game_id: int) -> BeautifulSoup:
    '''Fetch game data from BGG

    Args:
        game_id (int): numerical id of game on BGG

    Returns:
        (BeautifulSoup)
    '''
    params = {
        'stats': '1',
        'id': game_id
    }
    request_url = build_query('thing', params)
    return BeautifulSoup(requests.get(url=request_url).content, features='xml')


def fetch_search_results(query: str) -> BeautifulSoup:
    '''Fetch board game search results for given query

    Args:
        query (str): search query

    Returns:
        (BeautifulSoup)
    '''
    query = query.replace(' ', '+')
    url = f'https://boardgamegeek.com/xmlapi2/search?query={query}&type=boardgame'
    return BeautifulSoup(requests.get(url).content, features='xml')


def extract_ids_from_list(soup: BeautifulSoup) -> list:
    '''Extract IDs from listed results

    Args:
        soup (BeautifulSoup): List to extract ids from

    Returns:
        (list): list of all ids as str
    '''
    return [line.attrs['id'] for line in soup.find_all('item')]

def fetch_hotness() -> BeautifulSoup:
    '''Fetch list of Hottest 50 Games on BGG

    Returns:
        (BeautifulSoup)
    '''
    hot_list_url = 'https://boardgamegeek.com/xmlapi2/hot?boardgames'
    return BeautifulSoup(requests.get(hot_list_url).content, features='xml')

In [3]:
# Using Brass: Birmingham as our test game
brass_id = 224517

# Test build_query()
print('request url:', build_query('thing', {'id': brass_id, 'stats':'1'}))

# Test fetch_game()
soup = fetch_game(brass_id)

#print(soup.prettify())

request url: https://boardgamegeek.com/xmlapi2/thing?&id=224517&stats=1


In [4]:
# Extract some info from game query
title = soup.find('name', attrs={'type':'primary'})['value']

categories = [cat.attrs['value'] for cat in soup.find_all('link', type='boardgamecategory')]

mechanics = [mech.attrs['value'] for mech in soup.find_all('link', type='boardgamemechanic')]

avg_rating = soup.average['value']

avg_weight = soup.averageweight['value']

playing_time = soup.playingtime['value']

In [5]:
print(title)

Brass: Birmingham


In [6]:
# Get 'The Hotness' list [call api.fetch_hotness()]
hot_list_url = 'https://boardgamegeek.com/xmlapi2/hot?boardgames'
hot_soup = BeautifulSoup(requests.get(hot_list_url).content, features='xml')
#hot_soup.contents # show list

# Extract game ids from list
hot_game_ids = extract_ids_from_list(hot_soup)
len(hot_game_ids)

50

In [7]:
# Search query
search_soup = fetch_search_results('+')

# Extract game ids from list
game_ids = extract_ids_from_list(search_soup)

print(len(game_ids))

search_soup

500


<?xml version="1.0" encoding="utf-8"?>
<items termsofuse="https://boardgamegeek.com/xmlapi/termsofuse" total="500"> <item id="364866" type="boardgame">
<name type="primary" value='"SHOTGUN!": The Road Trip Game'/>
<yearpublished value="2022"/>
</item>
<item id="301429" type="boardgame">
<name type="primary" value="The #UpsideDownChallenge Game"/>
<yearpublished value="2020"/>
</item>
<item id="121039" type="boardgame">
<name type="primary" value="$ Fever"/>
<yearpublished value="1990"/>
</item>
<item id="10319" type="boardgame">
<name type="primary" value="The $1,000,000 Chance of a Lifetime Game"/>
<yearpublished value="1986"/>
</item>
<item id="317383" type="boardgame">
<name type="primary" value="The $10,000 Pyramid"/>
</item>
<item id="286876" type="boardgame">
<name type="primary" value="The $100,000 Pyramid DVD Game"/>
<yearpublished value="2006"/>
</item>
<item id="285001" type="boardgame">
<name type="primary" value="$100,000 Pyramid Game"/>
<yearpublished value="2019"/>
</item